# Data Cleaning
I will clean the Dataset in three Steps. Beforehand I already sorted the Dataset by id.

1. I run a couple of Validators on the Dataset to make sure only filled Data is used (Validate only Syntax) -> No Missing Data. When you are just looking for NaN-Values, you could also use data.dropnan() to remove all rows with NaN Elements, but I wanted to demonstrate my way of thinking here so I implemented it manually. Also you could extend the validators here to not just validate the value against NaN.

    -> No Missing Data
    - NoContent: Remove Data without Content at all (empty lines)
    - NoId: Remove data without an ID -> Make sure the Data is unique
    - NoName: Remove Data without FirstName or LastName => FullName is not necessarry, because you can alsways construct it
    - NoEmail: Remove Data without email -> You cannot contact People if they have no email
    - NoGender: Data where Gender is not Male or Female
    - NoAge: Remove Data where Age is not valid

2. I check the Data Set semantically (Analyze Data for semantic Errors) in the same corresponding validators from above.

    - CheckFullname: Fullname should be combination of Firstname and Lastname -> No Inconsistent Data
    - CheckAge: Only Age > 0 is allowed -> No Inconsistend Data "old" or incorrect Data "-78"
    - CheckGender: Only Gender Male or Female is allowed

3. I drop all duplicate Values -> No Duplications

In [1]:
# Used Imports
import pandas as pd
import math

In [2]:
# Declare general useful Functions
def isNaN(num):
    return pd.isna(num)

In [3]:
# Load and Display Dataset
data = pd.read_csv("https://assets.ctfassets.net/df3z1ez0ud42/1keo1CqBJJyJhltQx7g2n4/6e046ac756f26d36b18e9487251c0144/DB.csv")
data = data.sort_values(by=["id"])
data

,id,full_name,first_name,last_name,email,gender,age
0,1.0,Mariel Finnigan,Mariel,Finnigan,mfinnigan0@usda.gov,Female,60
1,2.0,Kenyon Possek,Kenyon,Possek,kpossek1@ucoz.com,Male,12
2,3.0,Lalo Manifould,Lalo,Manifould,lmanifould2@pbs.org,Male,26
3,4.0,Nickola Carous,Nickola,Carous,ncarous3@phoca.cz,Male,4
4,5.0,Norman Dubbin,Norman,Dubbin,ndubbin4@wikipedia.org,Male,17
5,6.0,Hasty Perdue,Hasty,Perdue,hperdue5@qq.com,NaN,77
6,7.0,Franz Castello,Franz,Castello,fcastello6@1688.com,Male,25
7,8.0,Jorge Tarney,Jorge,Tarney,jtarney7@ft.com,Male,77
8,9.0,Eunice Blakebrough,Eunice,Blakebrough,eblakebrough8@sohu.com,Female,45
9,10.0,Kristopher Frankcombe,Kristopher,Frankcombe,kfrankcombe9@slate.com,Male,old


In [4]:
# Initialize important Dataset Attributes and Constants for avoiding Magic Numbers in Validators
ID_INDEX = 0
FULLNAME_INDEX = 1
FIRSTNAME_INDEX = 2
LASTNAME_INDEX = 3
EMAIL_INDEX = 4
GENDER_INDEX = 5
AGE_INDEX = 6

In [5]:
# Syntax Validators and Sematic Validations
def noContent(row):
    return isNaN(row.all())

def noId(id):
    if isNaN(id):
        return True 

def noName(fullname, firstname, lastname):
    # Syntax
    if isNaN(firstname) or isNaN(lastname) or firstname == "" or lastname == "":
        return True
    # Semantics
    if fullname != (firstname + " " + lastname):
        return True

def noEmail(email):
    if isNaN(email) or email == "":
        return True

def noGender(gender):
    # Syntax
    if isNaN(gender) or gender == "":
        return True
    # Semantics
    if gender.lower() != "male" and gender.lower() != "female":
        return True

def noAge(age):
    # Syntax
    #print(age)
    #print(isNaN(age) or age == "" or not age.isnumeric())
    if isNaN(age) or age == "" or not age.isnumeric():
        return True
    # Semantics
    
    if int(age) < 0 or int(age) > 150:
        return True

def rowIsInvalid(row):
    if noContent(row):
        return True
    
    if noId(row[ID_INDEX]):
        return True

    if noName(row[FULLNAME_INDEX], row[FIRSTNAME_INDEX], row[LASTNAME_INDEX]):
        return True

    if noEmail(row[EMAIL_INDEX]):
        return True

    if noGender(row[GENDER_INDEX]):
        return True

    if noAge(row[AGE_INDEX]):
        return True 

    return False

# Collect all invalid Rows
invalidRows = set()
for index, row in data.iterrows():
    if rowIsInvalid(row) == True:
        invalidRows.add(index)

# Drop all collected invalid Rows
data = data.drop(list(invalidRows))

# Drop Duplicates
data.drop_duplicates({"full_name", "first_name", "last_name", "email", "gender", "age"}, keep=False, inplace=True)

# Cleaned Dataset
data

,id,full_name,first_name,last_name,email,gender,age
0,1.0,Mariel Finnigan,Mariel,Finnigan,mfinnigan0@usda.gov,Female,60
1,2.0,Kenyon Possek,Kenyon,Possek,kpossek1@ucoz.com,Male,12
2,3.0,Lalo Manifould,Lalo,Manifould,lmanifould2@pbs.org,Male,26
3,4.0,Nickola Carous,Nickola,Carous,ncarous3@phoca.cz,Male,4
4,5.0,Norman Dubbin,Norman,Dubbin,ndubbin4@wikipedia.org,Male,17
6,7.0,Franz Castello,Franz,Castello,fcastello6@1688.com,Male,25
7,8.0,Jorge Tarney,Jorge,Tarney,jtarney7@ft.com,Male,77
8,9.0,Eunice Blakebrough,Eunice,Blakebrough,eblakebrough8@sohu.com,Female,45
10,11.0,Palm Domotor,Palm,Domotor,pdomotora@github.io,Male,6
11,12.0,Luz Lansdowne,Luz,Lansdowne,llansdowneb@theguardian.com,Female,16


# Further Improvements
The Data can further be optimized by analyzing emails for example "sbovisd@webeden.co.uk" is no valid email.